In [ ]:
## SunnyX9
## 22.4.2024
## PreFer data challenge

In [ ]:
import sys
print(sys.version)
# needs to be 3.11.7 to match PreFer submission

In [ ]:
import pandas as pd

file_path = ''
df = pd.read_csv(file_path)

print(df.head())

In [ ]:
# filter to only include cases with outcome known: N 987

file_path = ''
outcome = pd.read_csv(file_path)

outcome = outcome.dropna(subset=['new_child'])

df_filt = df[df['nomem_encr'].isin(outcome['nomem_encr'])]

unique_cases = df_filt['nomem_encr'].nunique()

print("Number of unique cases in 'nomem_encr':", unique_cases)


In [ ]:
# drop some background vars
# how does our algorithm react to multicolleniarity?

columns_to_drop = ["positie", "lftdcat", "lftdhhh", "aantalhh", "woonvorm", "brutoink", "nettoink", "brutocat", "nettocat", "oplcat", "doetmee", "simpc", "netinc", "nettoink_f", "nettohh_f", "werving", "age_imp"] 
df_filt.drop(columns=columns_to_drop, axis=1, inplace=True)

df_filt.head()

In [ ]:
# correlation

import seaborn as sns
import matplotlib.pyplot as plt

corr = df_filt.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm',
            xticklabels=corr.columns, yticklabels=corr.columns,
            linewidths=.5, cbar_kws={"shrink": .5})

plt.title('Correlation Matrix')
plt.show()


In [ ]:
# into categories
# income vars separating 0 values as Q0, top 5% as Q5, and in between into quartiles Q1-Q4

cols_to_cat = ['brutoink_f', 'brutohh_f']
for column in cols_to_cat:
    df_filt[column + '_cat'] = 'Q0'  
    df_filt.loc[df[column] > 0, column + '_cat'] = pd.NA  
    df_filt.loc[df[column].isna(), column + '_cat'] = pd.NA

    non_zero_non_na_mask = (df_filt[column] > 0) & (~df_filt[column].isna())
    if non_zero_non_na_mask.any(): 
        quantiles = df_filt.loc[non_zero_non_na_mask, column].quantile([0.0, 0.25, 0.5, 0.75, 0.95, 1]).tolist()
        labels = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']  
        df_filt.loc[non_zero_non_na_mask, column + '_cat'] = pd.cut(df_filt.loc[non_zero_non_na_mask, column], 
                                                                     bins=quantiles, labels=labels, include_lowest=True)

print(df_filt['brutoink_f_cat'].value_counts(dropna=False))
print(df_filt['brutohh_f_cat'].value_counts(dropna=False))

cols_to_drop = ['brutoink_f', 'brutohh_f']
df_filt.drop(columns=cols_to_drop, axis=1, inplace=True)

In [ ]:
# into categories
# rest were already grouped into categories, just converting the type

columns_to_convert = ["wave", "aantalki", "partner", "burgstat", "woning", "belbezig", "oplzon", "oplmet", "sted", "birthyear_imp", "gender_imp", "migration_background_imp", "brutoink_f_cat", "brutohh_f_cat"]

for column in columns_to_convert:
    df_filt[column] = df_filt[column].astype('category')

print(df_filt.dtypes)

# now only nomem_encr & nohouse_encr are integers                  

In [ ]:
## NA per case 

unique_cases = df_filt.iloc[:, 0].unique()
total_variables = len(df_filt.columns) - 1

for case in unique_cases:

    df_case = df_filt[df_filt.iloc[:, 0] == case]
    
    total_instances = df_case.shape[0]
    
    missing_values = df_case.iloc[:, 1:].isnull().sum().sum()
    
    total_possible_values = total_instances * total_variables
    
    share_of_missing = (missing_values / total_possible_values) * 100
    
    print(f"case {case}: {missing_values} missing values out of {total_possible_values} possible values, {share_of_missing:.2f}% missing, {total_instances} instances")

# case = person
# missing values = per person
# instances = total rows per person

In [ ]:
# NA per predictor
for column in df_filt.columns[1:]:
    missing_count = df_filt[column].isnull().sum()
    print(f"{column} contains {missing_count} missing values")

# lots of NAs:
# gross and net income
# migration background
# level of urbanisation (sted)